In [6]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from torch_geometric.utils import dense_to_sparse
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
# 1. 导入 SummaryWriter
from torch.utils.tensorboard import SummaryWriter

# --- 实例化 SummaryWriter ---
# 这会创建一个 'runs/iris_fca_gnn_with_validation' 文件夹来存放日志
writer = SummaryWriter('../runs/iris_fca_gnn_with_validation')

# --- 数据加载与预处理 (与你的代码相同) ---

# a) 加载节点特征矩阵 X
features_path = '../data/iris.data.cleaned.csv'
x_numpy = np.loadtxt(features_path, delimiter=',')
x = torch.tensor(x_numpy, dtype=torch.float)

# b) 加载并稀疏化邻接矩阵 A+
adj_matrix_path = '../data/iris_A_plus.csv'
a_plus_numpy = np.loadtxt(adj_matrix_path, delimiter=',')
a_plus = torch.tensor(a_plus_numpy, dtype=torch.float)
print(f"原始邻接矩阵中的非零元素数: {torch.count_nonzero(a_plus)}")
threshold = 10
a_plus[a_plus <= threshold] = 0
a_plus.fill_diagonal_(0)
print(f"阈值化后邻接矩阵中的非零元素数: {torch.count_nonzero(a_plus)}")
edge_index, edge_attr = dense_to_sparse(a_plus)

# c) 加载标签 y
labels_path = '../data/iris.data'
column_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']
iris_df = pd.read_csv(labels_path, header=None, names=column_names)
species_labels = iris_df['species'].values
encoder = LabelEncoder()
y_numpy = encoder.fit_transform(species_labels)
y = torch.tensor(y_numpy, dtype=torch.long)

num_nodes = x.shape[0]
if num_nodes != len(y):
    print(f"警告: 特征矩阵的节点数 ({num_nodes}) 与标签数 ({len(y)}) 不匹配。")
    y = y[:num_nodes]
    print(f"已将标签截取为 {len(y)} 个以匹配节点数。")

# d) 创建 PyTorch Geometric 的 Data 对象
data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y)

# --- 2. 【修改】数据划分：训练(60%)、验证(20%)、测试(20%) ---
num_train = int(num_nodes * 0.6)
num_val = int(num_nodes * 0.2)
num_test = num_nodes - num_train - num_val

indices = torch.randperm(num_nodes)

# 创建训练、验证和测试的掩码 (mask)
data.train_mask = torch.zeros(num_nodes, dtype=torch.bool)
data.val_mask = torch.zeros(num_nodes, dtype=torch.bool)
data.test_mask = torch.zeros(num_nodes, dtype=torch.bool)

data.train_mask[indices[:num_train]] = True
data.val_mask[indices[num_train:num_train + num_val]] = True
data.test_mask[indices[num_train + num_val:]] = True


print("--- 数据加载完成 ---")
print(data)
print(f"训练节点数: {data.train_mask.sum().item()}")
print(f"验证节点数: {data.val_mask.sum().item()}") # 新增
print(f"测试节点数: {data.test_mask.sum().item()}")
print(f"类别数: {len(np.unique(y_numpy))}")


# --- GNN 模型定义 (与你的代码相同) ---
class GCN(torch.nn.Module):
    def __init__(self, num_node_features, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_node_features, 16)
        self.conv2 = GCNConv(16, num_classes)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return x

# --- 3. 【修改】训练与评估 ---
model = GCN(num_node_features=data.num_node_features, num_classes=len(np.unique(y_numpy)))
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

def train(epoch):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    writer.add_scalar('Loss/train', loss.item(), epoch)
    return loss.item()

def evaluate(epoch):
    """一个通用的评估函数，计算并记录所有数据集划分的准确率"""
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        pred = out.argmax(dim=1)
        
        # 计算训练集准确率
        correct_train = pred[data.train_mask] == data.y[data.train_mask]
        train_acc = int(correct_train.sum()) / int(data.train_mask.sum())
        
        # 计算验证集准确率
        correct_val = pred[data.val_mask] == data.y[data.val_mask]
        val_acc = int(correct_val.sum()) / int(data.val_mask.sum())
        
        # 计算测试集准确率
        correct_test = pred[data.test_mask] == data.y[data.test_mask]
        test_acc = int(correct_test.sum()) / int(data.test_mask.sum())

        # 记录到 TensorBoard
        writer.add_scalar('Accuracy/train', train_acc, epoch)
        writer.add_scalar('Accuracy/validation', val_acc, epoch) # 新增
        writer.add_scalar('Accuracy/test', test_acc, epoch)
        
        return train_acc, val_acc, test_acc

print("\n--- 开始训练 ---")
for epoch in range(1, 201):
    loss = train(epoch)
    
    # 在每个 epoch 后都进行评估
    train_acc, val_acc, test_acc = evaluate(epoch)
    
    if epoch % 20 == 0:
        # 在打印信息中加入验证集准确率
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}')

# 训练结束后关闭 writer
writer.close()

# 获取最后一个 epoch 的测试精度作为最终结果
final_test_acc = evaluate(200)[2] 
print(f'--- 训练完成 ---')
print(f'最终测试集准确率: {final_test_acc:.4f}')
print("\nTensorBoard 日志已写入 'runs' 文件夹。")

原始邻接矩阵中的非零元素数: 22500
阈值化后邻接矩阵中的非零元素数: 653
--- 数据加载完成 ---
Data(x=[150, 126], edge_index=[2, 653], edge_attr=[653], y=[150], train_mask=[150], val_mask=[150], test_mask=[150])
训练节点数: 90
验证节点数: 30
测试节点数: 30
类别数: 3

--- 开始训练 ---
Epoch: 020, Loss: 0.1793, Train Acc: 0.9778, Val Acc: 1.0000, Test Acc: 1.0000
Epoch: 040, Loss: 0.0518, Train Acc: 0.9889, Val Acc: 1.0000, Test Acc: 1.0000
Epoch: 060, Loss: 0.0487, Train Acc: 1.0000, Val Acc: 1.0000, Test Acc: 1.0000
Epoch: 080, Loss: 0.0359, Train Acc: 1.0000, Val Acc: 1.0000, Test Acc: 1.0000
Epoch: 100, Loss: 0.0571, Train Acc: 1.0000, Val Acc: 1.0000, Test Acc: 1.0000
Epoch: 120, Loss: 0.0210, Train Acc: 1.0000, Val Acc: 1.0000, Test Acc: 1.0000
Epoch: 140, Loss: 0.0557, Train Acc: 1.0000, Val Acc: 1.0000, Test Acc: 1.0000
Epoch: 160, Loss: 0.0419, Train Acc: 1.0000, Val Acc: 1.0000, Test Acc: 1.0000
Epoch: 180, Loss: 0.0255, Train Acc: 1.0000, Val Acc: 1.0000, Test Acc: 1.0000
Epoch: 200, Loss: 0.0133, Train Acc: 1.0000, Val Acc: 1.0000, Te